In [130]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [131]:
import glob
import cv2

images = [cv2.imread(file) for file in glob.glob("./images/*.jpg")]
imgangles = np.genfromtxt('angles.csv', delimiter=',', dtype=np.double)
print(imgangles)

[[-1.56648636 -1.06202924 -3.13790846]
 [-1.56774724 -0.8811692  -3.13930869]
 [-1.56682122 -0.69766629 -3.13910937]
 [-1.56623077 -0.51866692 -3.1394031 ]
 [-1.56703949 -0.34525266 -3.14039803]
 [-1.56834388 -0.17173445 -3.14125061]
 [-1.56906998 -0.00392006  3.14151931]
 [-1.56857049  0.17508206  3.14111209]
 [-1.567204    0.35158312  3.14028144]
 [-1.56640077  0.53479695  3.13926697]
 [-1.56667125  0.69997364  3.1388278 ]
 [-1.56743503  0.83966601  3.13900137]]


In [132]:
imgres = len(images[0])
u0 = int(imgres/2 + 0.5)
v0 = int(imgres/2 + 0.5)
fov = 60

In [133]:
# Pre-processing
grays = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]
grays = [cv2.medianBlur(gray,5) for gray in grays]

In [141]:
centers = []
for gray, imgangle in zip(grays, imgangles[:,1]):
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, gray.shape[0] / 8, param1=30,param2=15,minRadius=10,maxRadius=0)
    if circles is not None: # Convert it to world frame coordinates
        for circle in circles[0,:]: # What??
            radius = circle[2]
            if circle[0] < radius or circle[1] < radius or circle[0] > imgres-radius or circle[1] > imgres-radius:
                continue
            k = radius / (1.5 / 2) # Pixels per meter at the range of this circle
            xmax = imgres / (2.0*k) # Half the screen in meters at this range
            zc = xmax / tan(pi/6)
            xc = (circle[0] - u0) / k
            xw = zc*cos(imgangle+pi/2)-xc*sin(imgangle+pi/2)
            yw = zc*sin(imgangle+pi/2)-xc*cos(imgangle+pi/2)
            centers.append((xc,zc))
print(centers)

[(-1.8806990009203972, 5.054008201057986), (2.252435120729598, 5.398600053422637), (1.2369791994354247, 5.773502844841851), (0.030587276074000646, 2.712510270735617), (0.9672618754776887, 6.598288590972919), (-0.5575980322644847, 2.7169424093749948), (-0.26442308123373504, 3.5529247913948447), (0.015060241194579362, 6.677786348352834), (0.7416044987238557, 6.204361278364337), (0.2284946267800639, 5.959744795701594), (-0.8941281017408256, 5.91732652204026), (-2.0273852136668613, 5.8755081982073945), (0.6668410148276008, 3.4785958196772016), (0.04175946405969075, 3.7032710885494207), (-0.6468340719103762, 3.6304995697917817), (1.9500000000000002, 6.046431910058627)]
